<a href="https://colab.research.google.com/github/Santiago8888/Kin/blob/dev/densenet-121-e1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"dzetaa","key":"00431f0f6a6a7725bdbe7b053b8ce916"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition -p /content

In [0]:
!unzip \*.zip


In [0]:
! mkdir training
! mkdir testing

In [0]:
! mv train training/train
! mv test testing/test

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [0]:
import os
 
dirpath = os.getcwd()
# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(dirpath + '/training', transform=train_transforms)
test_data = datasets.ImageFolder(dirpath + '/testing', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [0]:
model = models.densenet121(pretrained=True)

In [0]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [14]:
import time
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 6.139 seconds
Device = cuda; Time per batch: 0.008 seconds


In [15]:
import time
for device in ['cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cuda; Time per batch: 0.008 seconds


In [0]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [0]:
for ii, (inputs, labels) in enumerate(trainloader):
  print (ii, labels, labels.size())
print (ii*64)
  


In [0]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

In [0]:
!ls training/train


In [62]:
i = 0
j = 0
k = 0
for file in os.listdir('training/train'):
  if 'cat' in file:
#    if i < 11250:
      i += 1
#      os.rename('training/train/'+file, 'train/cats/'+file)
  elif 'dog' in file:
#    if i < 11250:
      j += 1
#      os.rename('training/train/'+file, 'train/dogs/'+file)      
  k += 1
        
  
print(i, j, k)


0 0 2


In [61]:
!ls

sample_data	       testing	 train	   train.zip
sample_submission.csv  test.zip  training


In [0]:
! mkdir train/cats
! mkdir train/dogs

In [72]:
!ls

test.zip


In [0]:
os.rename("training/train/cat.11752.jpg", "training/train/cts/cat.11752.jpg")

In [0]:
!unzip \*.zip


In [77]:
!ls

mv: cannot stat 'test': No such file or directory


In [0]:
!rm -r test.zip

In [83]:
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition -p /content

 98% 266M/271M [00:02<00:00, 160MB/s]
100% 271M/271M [00:02<00:00, 137MB/s]
 98% 532M/544M [00:03<00:00, 212MB/s]
100% 544M/544M [00:03<00:00, 168MB/s]
  0% 0.00/111k [00:00<?, ?B/s]
100% 111k/111k [00:00<00:00, 114MB/s]


In [0]:
!rm test.zip

In [0]:
!unzip \*.zip


In [0]:
!rm train.zip

In [0]:
!mv train images

In [0]:
!mkdir test
!mkdir train

In [95]:
!ls

images	test  train


In [97]:
!mkdir train/cats
!mkdir train/dogs

!mkdir test/cats
!mkdir test/dogs

mkdir: cannot create directory ‘train/cats’: File exists
mkdir: cannot create directory ‘train/dogs’: File exists
mkdir: cannot create directory ‘test/cats’: File exists


In [101]:
cat_counter = 0
dog_counter = 0
for file in os.listdir('images'):
  if 'cat' in file:
    if cat_counter < 11250:
      os.rename('images/' + file, 'train/cats/' + file)
      cat_counter += 1
    else:
      os.rename('images/' + file, 'test/cats/' + file)
  elif 'dog' in file:
    if dog_counter < 11250:
      os.rename('images/' + file, 'train/dogs/' + file)
      dog_counter +=1
    else:
      os.rename('images/' + file, 'test/dogs/' + file)

print(cat_counter, dog_counter)

11250 11250


In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [0]:
import os
 
dirpath = os.getcwd()
# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder('train', transform=train_transforms)
test_data = datasets.ImageFolder('test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [0]:
for ii, (inputs, labels) in enumerate(testloader):
  print (ii, labels, labels.size())
print (ii*64)

In [0]:
model = models.densenet121(pretrained=True)

In [0]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [109]:
import time
for device in ['cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cuda; Time per batch: 0.008 seconds


In [110]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 0.367.. Test loss: 0.119.. Test accuracy: 0.962
Epoch 1/1.. Train loss: 0.257.. Test loss: 0.106.. Test accuracy: 0.962
Epoch 1/1.. Train loss: 0.202.. Test loss: 0.099.. Test accuracy: 0.963
Epoch 1/1.. Train loss: 0.213.. Test loss: 0.072.. Test accuracy: 0.973
Epoch 1/1.. Train loss: 0.159.. Test loss: 0.106.. Test accuracy: 0.961
Epoch 1/1.. Train loss: 0.212.. Test loss: 0.093.. Test accuracy: 0.964
Epoch 1/1.. Train loss: 0.169.. Test loss: 0.079.. Test accuracy: 0.969
Epoch 1/1.. Train loss: 0.174.. Test loss: 0.058.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.193.. Test loss: 0.055.. Test accuracy: 0.978
Epoch 1/1.. Train loss: 0.129.. Test loss: 0.065.. Test accuracy: 0.973
Epoch 1/1.. Train loss: 0.125.. Test loss: 0.053.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.125.. Test loss: 0.055.. Test accuracy: 0.977
Epoch 1/1.. Train loss: 0.161.. Test loss: 0.064.. Test accuracy: 0.973
Epoch 1/1.. Train loss: 0.187.. Test loss: 0.052.. Test accuracy

In [0]:
torch.save(model.state_dict(), 'densenet-121-1e.pth')

In [112]:
ResNeXt-101-32x8d

SyntaxError: ignored

In [113]:
model = models.ResNeXt-101-32x8d(pretrained=True)

SyntaxError: ignored

In [0]:
model2 = models.densenet121(pretrained=True)

In [0]:
# Freeze parameters so we don't backprop through them
for param in model2.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model2.classifier = classifier

In [0]:
epochs = 2
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model2.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model2.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model2.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model2.train()

Epoch 1/2.. Train loss: 0.708.. Test loss: 0.727.. Test accuracy: 0.367


In [0]:
import time
for device in ['cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model2.to(device)
